In [2]:
# Hücre 1: Kütüphaneler
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib # Modeli kaydetmek için
import warnings

warnings.filterwarnings('ignore')

# Hücre 2: Veri Yükleme ve Feature Engineering (Önceki adımın aynısı)
df = pd.read_csv(r'C:\Users\Sedat\PycharmProjects\mining-quality-prediction\data\MiningProcess_Flotation_Plant_Database.csv', decimal=',')
df.columns = [col.replace(' ', '_').replace('%_', '').replace('(', '').replace(')', '') for col in df.columns]
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date')

# Feature Engineering (Lag ve Rolling)
input_cols = [col for col in df.columns if col not in ['date', 'Silica_Concentrate']]
for col in input_cols:
    df[f'{col}_Rolling_Mean'] = df[col].rolling(window=5).mean()
    df[f'{col}_Lag1'] = df[col].shift(1)

df = df.dropna()

# Veriyi Ayırma
X = df.drop(['date', 'Silica_Concentrate'], axis=1)
y = df['Silica_Concentrate']

# XGBoost için validasyon setine de ihtiyacımız var
# %80 Train, %10 Validation, %10 Test yapacağız
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, shuffle=False, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.11, shuffle=False, random_state=42)
# 0.11 matematiksel olarak kalanın %10'una denk gelir yaklaşık

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Hücre 3: Optuna ile Hiperparametre Arama
# Optuna, en iyi parametreleri bulmak için yüzlerce deneme yapar.

def objective(trial):
    # Denenecek parametre aralıkları
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'tree_method': 'hist', # Hızlandırmak için
        'device': 'cpu' # Eğer GPU varsa 'cuda' yapabilirsin
    }

    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))

    return rmse

print("Optuna optimizasyonu başlıyor... (Bu işlem 2-3 dakika sürebilir)")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20) # 20 deneme yapacağız (Daha iyi sonuç için 50-100 yapılabilir)

print("\nEn İyi Parametreler:")
print(study.best_params)

# Hücre 4: En İyi Parametrelerle Final Modeli Eğitme
best_params = study.best_params
final_model = xgb.XGBRegressor(**best_params, random_state=42)

print("\nFinal model eğitiliyor...")
final_model.fit(X_train, y_train)

# Test Seti Üzerinde Sonuçlar
y_pred = final_model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
final_r2 = r2_score(y_test, y_pred)

print("--- XGBoost Optuna Sonuçları ---")
print(f"Final R2 Score: {final_r2:.4f}")
print(f"Final RMSE: {final_rmse:.4f}")

# Hücre 5: Modeli Kaydet (Deployment için lazım olacak)
joblib.dump(final_model, '../models/final_xgboost_model.pkl')
print("Model '../models/' klasörüne kaydedildi.")

[I 2025-11-19 22:18:11,678] A new study created in memory with name: no-name-8b5f3bb4-1a66-4c84-ad05-4176c140d28a


Train: (590696, 66), Val: (73008, 66), Test: (73745, 66)
Optuna optimizasyonu başlıyor... (Bu işlem 2-3 dakika sürebilir)


[I 2025-11-19 22:19:29,412] Trial 0 finished with value: 0.7764805364391663 and parameters: {'n_estimators': 770, 'max_depth': 8, 'learning_rate': 0.1772136557187996, 'subsample': 0.6820906497847694, 'colsample_bytree': 0.9732893369499102, 'reg_alpha': 4.101230655582514, 'reg_lambda': 9.723449417030173}. Best is trial 0 with value: 0.7764805364391663.
[I 2025-11-19 22:19:43,768] Trial 1 finished with value: 0.7112305960866042 and parameters: {'n_estimators': 209, 'max_depth': 5, 'learning_rate': 0.02610586158751911, 'subsample': 0.7544633968375494, 'colsample_bytree': 0.8104822462902593, 'reg_alpha': 4.216537975508146, 'reg_lambda': 3.846481680451308}. Best is trial 1 with value: 0.7112305960866042.
[I 2025-11-19 22:20:39,061] Trial 2 finished with value: 0.7713035046026837 and parameters: {'n_estimators': 873, 'max_depth': 6, 'learning_rate': 0.04899940035075583, 'subsample': 0.7117565072443119, 'colsample_bytree': 0.6653999446656195, 'reg_alpha': 4.603973627671058, 'reg_lambda': 8.73


En İyi Parametreler:
{'n_estimators': 814, 'max_depth': 3, 'learning_rate': 0.04391546030163544, 'subsample': 0.7072420661612152, 'colsample_bytree': 0.6874523771884891, 'reg_alpha': 2.6649955595443497, 'reg_lambda': 8.344839349580123}

Final model eğitiliyor...
--- XGBoost Optuna Sonuçları ---
Final R2 Score: 0.6785
Final RMSE: 0.6715
Model '../models/' klasörüne kaydedildi.
